## Chapter 18: Working with Real‑Time Features (SignalR)

Traditional web applications are built on the request‑response model: the client asks, the server answers. But many modern applications require real‑time functionality—live dashboards, instant notifications, collaborative editing, or chat systems—where the server needs to push data to clients as soon as it’s available. **SignalR** is a library for ASP.NET Core that simplifies adding real‑time web functionality. It handles connection management, supports multiple transport protocols (WebSockets, Server‑Sent Events, Long Polling), and provides a high‑level API for calling methods on clients from the server and vice versa. In this chapter, you’ll learn how to integrate SignalR into your ASP.NET Core application, build a simple chat system, implement real‑time notifications, understand scaling considerations, and secure your connections.

### 18.1 Introduction to WebSockets

Before SignalR, real‑time web communication was difficult. The most efficient transport is **WebSockets**, which provides full‑duplex communication over a single TCP connection. However, not all clients or network configurations support WebSockets. SignalR abstracts this away: it chooses the best available transport and falls back to older techniques (Server‑Sent Events, Long Polling) when necessary. As a developer, you work with a unified API regardless of the underlying transport.

SignalR automatically manages connections, reconnections, and scaling. You write server code that defines **hubs**—high‑level pipelines that clients and servers can call. Clients (browser, mobile, desktop) use a SignalR client library to connect to these hubs and invoke methods or listen for messages.

### 18.2 Setting up SignalR Hubs

#### Adding SignalR to Your Project

First, install the SignalR server package in your ASP.NET Core project:

```bash
dotnet add package Microsoft.AspNetCore.SignalR
```

For client‑side development, you’ll also need the SignalR JavaScript client (or a .NET client for other applications).

#### Creating a Hub

A **hub** is a class that inherits from `Hub`. It contains methods that clients can call, and it can call methods on connected clients.

```csharp
using Microsoft.AspNetCore.SignalR;
using System.Threading.Tasks;

namespace MyApp.Hubs
{
    public class ChatHub : Hub
    {
        public async Task SendMessage(string user, string message)
        {
            // Broadcast the message to all connected clients
            await Clients.All.SendAsync("ReceiveMessage", user, message);
        }

        // Called when a new client connects
        public override async Task OnConnectedAsync()
        {
            await Clients.All.SendAsync("UserConnected", Context.ConnectionId);
            await base.OnConnectedAsync();
        }

        // Called when a client disconnects
        public override async Task OnDisconnectedAsync(Exception exception)
        {
            await Clients.All.SendAsync("UserDisconnected", Context.ConnectionId);
            await base.OnDisconnectedAsync(exception);
        }
    }
}
```

- `Clients.All.SendAsync` sends a message to all connected clients. The first parameter is the method name clients must handle (e.g., `ReceiveMessage`). Additional parameters are passed as arguments.
- `Context.ConnectionId` is a unique identifier for the connected client.
- Overriding `OnConnectedAsync` and `OnDisconnectedAsync` lets you react to connection events.

#### Registering SignalR in the Pipeline

In `Program.cs`, add SignalR to the services and map the hub endpoint:

```csharp
var builder = WebApplication.CreateBuilder(args);

builder.Services.AddControllersWithViews();
builder.Services.AddSignalR(); // Add SignalR services

var app = builder.Build();

app.UseHttpsRedirection();
app.UseStaticFiles();
app.UseRouting();

app.UseAuthentication();
app.UseAuthorization();

app.MapControllers();
app.MapHub<ChatHub>("/chathub"); // Map the hub to a URL

app.Run();
```

The hub will be accessible at `/chathub`. Clients connect to this URL using the SignalR client.

### 18.3 Sending Messages from the Server to Clients

The `Clients` property gives you access to different groups of clients:

- `Clients.All` – all connected clients.
- `Clients.Caller` – the client that invoked the current hub method.
- `Clients.Others` – all clients except the caller.
- `Clients.Client(connectionId)` – a specific client by connection ID.
- `Clients.Group(groupName)` – all clients in a named group.

#### Example: Private Message

You could extend the chat to support private messages if you know the recipient’s connection ID.

```csharp
public async Task SendPrivateMessage(string targetConnectionId, string message)
{
    await Clients.Client(targetConnectionId).SendAsync("ReceivePrivateMessage", Context.ConnectionId, message);
}
```

#### Strongly‑Typed Hubs

Instead of using magic strings for method names, you can define an interface for client methods and use a strongly‑typed hub.

```csharp
public interface IChatClient
{
    Task ReceiveMessage(string user, string message);
    Task UserConnected(string connectionId);
    Task UserDisconnected(string connectionId);
}

public class ChatHub : Hub<IChatClient>
{
    public async Task SendMessage(string user, string message)
    {
        await Clients.All.ReceiveMessage(user, message);
    }
}
```

Now, when you call `Clients.All.ReceiveMessage(...)`, it’s strongly typed, and you get IntelliSense. The client must implement methods with matching names.

### 18.4 Building a Live Chat or Notification System

Let’s build a simple chat application using SignalR.

#### Server‑Side Hub

We’ll use the `ChatHub` class from above.

#### Client‑Side (JavaScript)

Create an HTML page with a simple chat interface.

```html
<!DOCTYPE html>
<html>
<head>
    <title>SignalR Chat</title>
    <style>
        .message { margin: 5px; padding: 5px; background: #f0f0f0; }
        .own { background: #d9edf7; text-align: right; }
    </style>
</head>
<body>
    <div id="messages"></div>
    <input type="text" id="userInput" placeholder="Your name" />
    <input type="text" id="messageInput" placeholder="Type a message" />
    <button onclick="sendMessage()">Send</button>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/microsoft-signalr/8.0.0/signalr.min.js"></script>
    <script>
        const connection = new signalR.HubConnectionBuilder()
            .withUrl("/chathub")
            .configureLogging(signalR.LogLevel.Information)
            .build();

        connection.on("ReceiveMessage", (user, message) => {
            const msgDiv = document.createElement("div");
            msgDiv.classList.add("message");
            msgDiv.innerHTML = `<strong>${user}:</strong> ${message}`;
            document.getElementById("messages").appendChild(msgDiv);
        });

        connection.on("UserConnected", (connectionId) => {
            const msgDiv = document.createElement("div");
            msgDiv.innerHTML = `<em>User ${connectionId} connected</em>`;
            document.getElementById("messages").appendChild(msgDiv);
        });

        connection.on("UserDisconnected", (connectionId) => {
            const msgDiv = document.createElement("div");
            msgDiv.innerHTML = `<em>User ${connectionId} disconnected</em>`;
            document.getElementById("messages").appendChild(msgDiv);
        });

        connection.start().catch(err => console.error(err.toString()));

        function sendMessage() {
            const user = document.getElementById("userInput").value;
            const message = document.getElementById("messageInput").value;
            connection.invoke("SendMessage", user, message).catch(err => console.error(err.toString()));
            document.getElementById("messageInput").value = "";
        }
    </script>
</body>
</html>
```

- The client creates a connection to `/chathub`.
- `connection.on` registers handlers for server‑side calls (e.g., `ReceiveMessage`).
- `connection.start()` begins the connection.
- `connection.invoke` calls a server hub method (`SendMessage`).

#### Testing the Chat

Run your application, open multiple browser tabs, and you’ll see messages broadcast to all tabs.

#### Adding a Simple Notification System

You can also push notifications from the server outside of hub methods—for example, from a controller action.

```csharp
public class NotificationsController : ControllerBase
{
    private readonly IHubContext<ChatHub> _hubContext;

    public NotificationsController(IHubContext<ChatHub> hubContext)
    {
        _hubContext = hubContext;
    }

    [HttpPost("notify")]
    public async Task<IActionResult> Notify([FromBody] string message)
    {
        // Broadcast to all clients
        await _hubContext.Clients.All.SendAsync("ReceiveNotification", message);
        return Ok();
    }
}
```

The `IHubContext<T>` allows you to send messages from outside a hub, e.g., from background services, controllers, or other parts of your application.

### 18.5 Scaling SignalR Across Multiple Servers

SignalR works perfectly on a single server. However, when you scale out to multiple servers (a web farm), you need a **backplane** to ensure messages reach all connected clients regardless of which server they are connected to. Common backplane options:

- **Azure SignalR Service** – A fully managed service that handles scaling.
- **Redis** – Uses Redis pub/sub to broadcast messages.
- **SQL Server** – Uses SQL Server as a message broker (less common).

#### Setting Up Redis Backplane

Install the package:

```bash
dotnet add package Microsoft.AspNetCore.SignalR.StackExchangeRedis
```

Then in `Program.cs`:

```csharp
builder.Services.AddSignalR()
    .AddStackExchangeRedis("your-redis-connection-string");
```

Now, when you scale out, SignalR uses Redis to distribute messages to all servers.

### 18.6 Security: Authentication and Authorization

SignalR integrates with ASP.NET Core’s authentication system. You can use `[Authorize]` on hubs or hub methods, and access user information via `Context.User`.

#### Authenticating Users

When a client connects, it can pass an access token (for JWT) or use cookies (for cookie authentication). In the JavaScript client, you can add the token to the connection options:

```javascript
const connection = new signalR.HubConnectionBuilder()
    .withUrl("/chathub", { accessTokenFactory: () => yourToken })
    .build();
```

On the server, you can protect the hub:

```csharp
[Authorize]
public class ChatHub : Hub
{
    public override async Task OnConnectedAsync()
    {
        var userName = Context.User.Identity.Name;
        await Clients.All.SendAsync("UserConnected", userName);
    }
}
```

#### Passing User Information to Clients

When you call a client method, you can include the username instead of the connection ID.

#### Restricting Access to Groups

You can add users to groups based on roles or other criteria. For example, only admins can join the “Admins” group.

```csharp
[Authorize(Roles = "Admin")]
public async Task JoinAdminGroup()
{
    await Groups.AddToGroupAsync(Context.ConnectionId, "Admins");
}
```

### 18.7 Handling Connection Lifetime and State

SignalR automatically reconnects if the connection drops. The JavaScript client can be configured to automatically reconnect:

```javascript
const connection = new signalR.HubConnectionBuilder()
    .withUrl("/chathub")
    .withAutomaticReconnect()
    .build();
```

You can also handle reconnection events:

```javascript
connection.onreconnecting(error => { ... });
connection.onreconnected(connectionId => { ... });
```

On the server, you can use `OnConnectedAsync` and `OnDisconnectedAsync` to track users and maintain state (e.g., in a database). However, be aware that these methods are called for every reconnect, which may happen frequently. It’s often better to maintain user presence through a separate mechanism.

### 18.8 Advanced Topics: Streaming, Client Results, and More

SignalR supports more than just simple messaging:

- **Streaming:** Server can stream data to clients, and clients can stream to the server. Useful for large data sets or real‑time progress updates.
- **Client results:** You can call a client method and wait for a return value.
- **Hub filters:** Similar to middleware for hubs, allowing cross‑cutting concerns like logging, error handling, or metrics.

#### Example: Streaming Data from Server to Client

Server method:

```csharp
public async IAsyncEnumerable<int> StreamingCounter(int count, int delay)
{
    for (int i = 0; i < count; i++)
    {
        yield return i;
        await Task.Delay(delay);
    }
}
```

Client (JavaScript):

```javascript
const stream = connection.stream("StreamingCounter", 10, 500);
stream.subscribe({
    next: item => console.log(item),
    error: err => console.error(err),
    complete: () => console.log("Stream completed")
});
```

### 18.9 Testing SignalR Hubs

You can unit test hub methods by creating an instance of the hub with mocked `HubCallerContext` and `IHubCallerClients`. Use libraries like Moq to simulate clients and connections.

```csharp
[Fact]
public async Task SendMessage_BroadcastsToAllClients()
{
    // Arrange
    var mockClients = new Mock<IHubCallerClients<IChatClient>>();
    var mockClientProxy = new Mock<IChatClient>();
    mockClients.Setup(clients => clients.All).Returns(mockClientProxy.Object);

    var hub = new ChatHub { Clients = mockClients.Object };

    // Act
    await hub.SendMessage("User", "Hello");

    // Assert
    mockClientProxy.Verify(
        client => client.ReceiveMessage("User", "Hello"),
        Times.Once);
}
```

### 18.10 Putting It All Together: Real‑Time Dashboard Example

Imagine a dashboard that shows live sales data. You can push updates from the server whenever a new order is placed.

**SalesHub.cs**

```csharp
public class SalesHub : Hub
{
    public async Task UpdateDashboard(SaleData data)
    {
        await Clients.All.SendAsync("ReceiveSale", data);
    }
}
```

**OrderController.cs** (after placing an order)

```csharp
[HttpPost]
public async Task<IActionResult> PlaceOrder(Order order)
{
    // Save order
    // ...

    var saleData = new SaleData
    {
        Product = order.Product,
        Amount = order.Total,
        Time = DateTime.UtcNow
    };
    await _hubContext.Clients.All.SendAsync("ReceiveSale", saleData);

    return Ok();
}
```

The client listens for `ReceiveSale` and updates the dashboard in real time.

### Summary

In this chapter, you’ve learned how to add real‑time functionality to your ASP.NET Core applications using SignalR:

- **SignalR abstracts** the complexities of real‑time communication, automatically selecting the best transport.
- **Hubs** are the central pattern: they expose methods that clients can call, and can call methods on clients.
- You can send messages to **all clients, specific clients, or groups**.
- **IHubContext** lets you send messages from outside a hub (e.g., from controllers or background services).
- For **scaling** across multiple servers, you need a backplane like Redis or Azure SignalR Service.
- **Authentication and authorization** integrate seamlessly with ASP.NET Core’s security.
- Advanced features like **streaming** and **client results** open up even more possibilities.

SignalR is the go‑to solution for any real‑time requirement in ASP.NET Core. With the knowledge from this chapter, you can build live dashboards, chat applications, collaborative tools, and instant notification systems.

**Exercise:**

1. Add SignalR to your existing project. Create a simple hub that broadcasts a message when a new product is added (trigger from your `ProductsController` after successful creation).
2. Build a small chat page (HTML + JavaScript) that connects to your hub and allows users to send messages.
3. Implement a “typing” indicator: when a user types in the message box, send a notification to others that they are typing.
4. Secure your hub so that only authenticated users can send messages. Use either cookie authentication (if MVC) or JWT (if API).
5. (Optional) If you have multiple instances running, set up a Redis backplane and test scaling.

In the next chapter, **"Background Tasks and Hosted Services,"** you’ll learn how to run long‑running or scheduled tasks in the background, independent of HTTP requests. This is essential for jobs like sending emails, cleaning up files, or processing queues.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='17. clean_architecture_domain_driven_design.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='19. background_tasks_and_hosted_services.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
